In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from pathlib import Path
import lightning as L
import torch as t
from data import EOT_TOKEN, NamesDataset
from model import CharLangNet
from dataclasses import dataclass

In [24]:
DATAROOT = Path.home() / "mldata" / "makemore" 
RUNROOT = Path.home() / "mlruns" / "makemore" / "names"

In [4]:
# Somehow figure out how to get this from the checkpoint's metadata
context_len = 3

In [5]:
dataset = NamesDataset(DATAROOT / "names.txt", context_len)

In [6]:
ckpt_path = RUNROOT / "Makemore2Names"/"yo62km9o"/"checkpoints"/"epoch=9-step=71300.ckpt"
lang_net = CharLangNet.load_from_checkpoint(ckpt_path)

In [7]:
eot_tok_idx = dataset.idx_of(EOT_TOKEN)
gen_name = []
context = [eot_tok_idx] * context_len
predicted_idx = None

In [8]:
t.no_grad()

In [10]:
input = t.tensor(context).unsqueeze(0).to(lang_net.device)
input.shape

torch.Size([1, 3])

In [11]:
logits = lang_net.model.forward(input)
logits.shape

torch.Size([1, 27])

In [12]:
logits = logits.squeeze()
logits.shape

torch.Size([27])

In [13]:
probs = t.softmax(logits, 0)

In [14]:
print(probs.sum())
print(t.argmin(logits), t.argmax(logits))
print(t.argmin(probs), t.argmax(probs))

tensor(1., device='mps:0', grad_fn=<SumBackward0>)
tensor(0, device='mps:0') tensor(1, device='mps:0')
tensor(0, device='mps:0') tensor(1, device='mps:0')


In [16]:
predicted_idx = t.multinomial(probs, 1)[0].item()
predicted_idx

18

In [18]:
char = dataset.char_at(predicted_idx)
char

'r'

In [26]:
%reset

In [28]:
from pathlib import Path
import lightning as L
import torch as t
from data import EOT_TOKEN, NamesDataset
from model import CharLangNet

DATAROOT = Path.home() / "mldata" / "makemore" 
RUNROOT = Path.home() / "mlruns" / "makemore" / "names"
CONTEXT_LEN = 3


In [ ]:
ckpt_path = RUNROOT/"Makemore2Names"/"6116xp8k"/"checkpoints"/"epoch=99-step=713000.ckpt"
lang_net = CharLangNet.load_from_checkpoint(ckpt_path)
dataset = NamesDataset(DATAROOT / "names.txt", context_len)

In [58]:
@dataclass
class DebugInfo:
    prediction: tuple[list[int], int]
    rank: int
    probs: list[float] = None

In [64]:
def gen_name(vocab: NamesDataset, lang_net: CharLangNet) -> tuple[str, list[DebugInfo]]:
    eot_tok_idx = vocab.idx_of(EOT_TOKEN)
    name: list[str] = []
    context: list[int] = [eot_tok_idx] * CONTEXT_LEN
    predicted_idx = None
    dbg: list[DebugInfo] = []
    with t.no_grad():
        while predicted_idx != eot_tok_idx and len(name) < 10:
            input = t.tensor(context).unsqueeze(0).to(lang_net.device)
            logits = lang_net.model.forward(input).squeeze()
            probs = t.softmax(logits, 0)
            predicted_idx = t.multinomial(probs, 1)[0].item()

            vals, idxs = t.sort(probs, descending=True)
            rank = t.nonzero(idxs == predicted_idx).squeeze()
            dbg.append(DebugInfo(
                # probs=probs,
                prediction=(context, predicted_idx),
                rank=rank
            ))

            predicted_char = vocab.char_at(predicted_idx)
            name.append(predicted_char)
            context = context[1:] + [predicted_idx]
    return "".join(name), dbg

In [65]:
name, dbg = gen_name(dataset, lang_net)
print(name)

mailaiki.


In [67]:
for _ in range(10):
    name, _ = gen_name(dataset, lang_net)
    print(name)

martinaya.
mydie.
ausy.
ano.
kaishow.
anmellen.
ykol.
adraian.
aney.
javionthla
